In [1]:
library(pedigreemm)
library(MASS)

Loading required package: lme4

Loading required package: Matrix



In [15]:

data_manipulation <- function(data) {
    id <- c(as.character(data$ID),
            as.character(data$DamID),
            as.character(data$SireID))
    id <- unique(id)
    id <- id[id != "0"]
    new_data <- data.frame("ID" = id)
    new_data <- merge(new_data, data, by = "ID", all.x = TRUE)
    new_data <- new_data[, c(1,2,3,5)]
    new_data <- new_data[order(new_data$Byear),]
    n <- nrow(new_data)
    new_data$newID <- 1:n
    data_tmp <- new_data[, c(1, 5)]
    colnames(data_tmp) <- c("SireID", "newSireID")
    new_data <- merge(new_data, data_tmp, by = "SireID", all.x = TRUE)
    data_tmp <- new_data[, c(2, 5)]
    colnames(data_tmp) <- c("DamID", "newDamID")
    new_data <- merge(new_data, data_tmp, by = "DamID", all.x = TRUE)
    new_data <- new_data[order(new_data$Byear),]
    new_data <- new_data[, c(-1, -2, -4)]
    new_data$newSireID[new_data$newSireID > new_data$newID &
                        !is.na(new_data$newSireID)] <- NA
    new_data$newDamID[new_data$newDamID > new_data$newID &
                        !is.na(new_data$newDamID)] <- NA
    return(new_data)
}
add_information_columns <- function(data, new_data) {
    new_data$y1 <- merge(new_data, data[data$ID %in% new_data$ID, ][, c("Y1", "ID")], by="ID")$Y1
    new_data$y2 <- merge(new_data, data[data$ID %in% new_data$ID, ][, c("Y2", "ID")], by="ID")$Y2
    new_data$sex <- merge(new_data, data[data$ID %in% new_data$ID, ][, c("Sex", "ID")], by="ID")$Sex
    new_data$byear <- merge(new_data, data[data$ID %in% new_data$ID, ][, c("Byear", "ID")], by="ID")$Byear
    new_dataSNP <- merge(new_data, data[data$ID %in% new_data$ID, -c(2,3,4,5,6,7)], by = "ID")
    new_dataSNP <- new_dataSNP[order(new_dataSNP$newID), ]
    return(new_dataSNP)
}

prepare_data <- function(data_path, size = 500) {
    data <- read.table(data_path, header = TRUE, sep = ";")
    new_data <- data_manipulation(data)
    new_dataSNP <- add_information_columns(data, new_data)
    new_dataSNP <- new_dataSNP[0:size, ]
    return(new_dataSNP)
}

In [16]:
newData <- prepare_data("data.csv")

In [17]:
get_freq_A <- function(newData) {
    first_marker <- which(colnames(newData) == "SNP1")
    data_snp <- newData[first_marker:ncol(newData)]
    freq_A <- numeric(ncol(data_snp))
    for (i in seq_len(ncol(data_snp))) {
        n <- length(data_snp[, i]) * 2
        A <- 0
        for (j in data_snp[, i]) {
            if (j == -1) {
                A <- A + 2
            }
            else if (j == 0) {
                A <- A + 1
            }
        }
        freq_A[i] <- A/n
    }
    return(freq_A)
}

In [18]:
freq_A <- get_freq_A(newData)


In [19]:
remove_low_MAF <- function(newData, freq_A) {
    first_marker <- which(colnames(newData) == "SNP1")
    for (i in first_marker:ncol(newData)){
        if (freq_A[i-first_marker + 1] < 0.05 | 1-freq_A[i-first_marker + 1] < 0.05) {
            newData <- newData[, -i]
        }
    }
    return(newData)
}

In [20]:
newData_new <- remove_low_MAF(newData, freq_A)

In [21]:
ped <- pedigree(sire=newData$newSireID, dam=newData$newDamID, label=newData$newID)
A <- as.matrix(getA(ped))

In [22]:
y1 <- newData_new$y1
y2 <- newData_new$y2
byear <- newData_new$byear
sex <- newData_new$sex

X <- matrix(0,500,3)
X[, 1] <- sex
X[, 2] <- -sex
X[, 3] <- byear

Z <- diag(500)

In [23]:
p2 <- get_freq_A(newData_new)
length(p2)

[1] 151

In [24]:
first_marker <- which(colnames(newData_new) == "SNP1")
snp_data <- newData_new[first_marker:ncol(newData_new)]
M = matrix(0, nrow = 500, ncol = length(p2))
for (i in 1:500) {
    for (j in seq_len(length(p2))) {
       if (snp_data[i, j] == -1) {
           M[i, j] = 2 - 2*p2[j]
       } else if (snp_data[i, j] == 0) {
           M[i, j] = 1 - 2*p2[j]
       } else {
           M[i, j] = - 2*p2[j]
       }
    }
}
G = M%*%t(M) / (2*sum(p2*(1-p2)))

In [25]:
mme = function(y, X, Z, A, sigma_a, sigma_e) {
    alpha = sigma_e / sigma_a
    invA = ginv(A)
    C = rbind(cbind(t(X)%*%X, t(X)%*%Z),
              cbind(t(Z)%*%X, t(Z)%*%Z+invA*c(alpha)))
    rhs = rbind(t(X)%*%y, t(Z)%*%y)
    invC = ginv(C)
    estimators = invC%*%rhs
    list(C = C, est = estimators)
}


In [26]:
EM = function(y, X, Z, A, sigma_a, sigma_e, output) {
  n = nrow(X)
  p = ncol(X) 
  q = nrow(A) 
  t = 1 #iteration number 1
  tmp = 0.1 #test for convergance
  while (tmp > 0.0001) {
    mme_new = mme(y, X, Z, A, sigma_a, sigma_e)
    C_new = ginv(mme_new$C)
    Ck = C_new[(p+1):(p+q), (p+1):(p+q)]
    mme2 = mme_new$est
    a = as.matrix(mme2[(p+1):(p+q)])
    sigma_a_new = (t(a)%*%ginv(A)%*%a + sum(diag(ginv(A)%*%Ck))*c(sigma_e))/q
    res = as.matrix(y-X%*%as.matrix(mme2[1:p]) - Z%*%as.matrix(mme2[(p+1):(p+q)]))
    X.tmp1 = cbind(X,Z) %*% C_new
    X.tmp2 = t(cbind(X,Z))
    sigma_e_new = (t(res)%*%res + sum(diag(X.tmp1%*%X.tmp2))*c(sigma_e))/n
    tmp = max(abs(sigma_a - sigma_a_new), abs(sigma_e - sigma_e_new))
    sigma_a = sigma_a_new
    sigma_e = sigma_e_new
    t = t + 1
    write.table(c(sigma_a,sigma_e,t, tmp), output, col.names = FALSE, row.names = FALSE, append = TRUE, quote = FALSE)
  }
  list(t = t, sigma_a = sigma_a, sigma_e = sigma_e)
}

In [27]:
results_y1 = EM(y1, X, Z, A, sigma_a = 0.3*var(y1), sigma_e = 0.7*var(y1), "Results_y1.csv")


In [18]:
results_y2 = EM(y1, X, Z, A, sigma_a = 0.3*var(y1), sigma_e = 0.7*var(y1), "Results_y2.csv")

In [28]:
mme2 = function(y, X, Z1, Z2, A, G, sigma_a, sigma_g, sigma_e) {
    alpha1 = sigma_e / sigma_a
    alpha2 = sigma_e / sigma_g
    invA = ginv(A)
    invG = ginv(G)
    C = rbind(cbind(t(X)%*%X, t(X)%*%Z1, t(X)%*%Z2),
              cbind(t(Z1)%*%X, t(Z1)%*%Z1+invA*c(alpha1), t(Z1)%*%Z2),
              cbind(t(Z2)%*%X, t(Z2)%*%Z1, t(Z2)%*%Z2 + invG*c(alpha2)))
    rhs = rbind(t(X)%*%y, t(Z1)%*%y, t(Z2)%*%y)
    invC = ginv(C)
    estimators = invC%*%rhs
    list(C = C, est = estimators)
}


In [22]:
(h2_y1 = results_y1$sigma_a / (results_y1$sigma_a + results_y1$sigma_e))

2.211236e-07


In [24]:
(h2_y2 = results_y2$sigma_a / (results_y2$sigma_a + results_y2$sigma_e))

2.211236e-07


In [32]:
Z2 <- data.matrix(snp_data)

In [33]:
Z2

,SNP1,SNP3,SNP4,SNP5,SNP6,SNP7,SNP8,SNP9,SNP10,SNP11,⋯,SNP155,SNP156,SNP157,SNP158,SNP159,SNP161,SNP163,SNP164,SNP165,SNP166
756,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,⋯,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1706,1,1,0,1,1,0,1,0,1,0,⋯,0,0,1,1,0,0,0,0,0,0
1724,-1,-1,-1,0,-1,-1,0,-1,0,-1,⋯,1,-1,-1,0,0,0,-1,-1,-1,-1
2121,-1,-1,-1,0,-1,-1,-1,-1,0,-1,⋯,0,-1,-1,0,0,0,-1,-1,-1,-1
3838,-1,1,-1,1,0,-1,0,-1,1,-1,⋯,0,-1,0,0,1,1,-1,-1,-1,-1
4662,-1,0,-1,1,0,-1,0,-1,0,-1,⋯,0,-1,0,1,0,1,-1,-1,-1,0
4912,-1,0,-1,0,0,-1,0,-1,0,-1,⋯,1,-1,0,1,0,0,-1,-1,-1,-1
5154,-1,-1,-1,0,-1,-1,0,-1,0,-1,⋯,0,-1,-1,0,0,0,-1,-1,-1,-1
5551,0,1,0,1,1,0,1,1,1,0,⋯,0,0,1,1,1,1,0,0,1,0
6043,0,0,-1,1,0,0,1,0,1,-1,⋯,1,0,0,0,0,1,0,0,0,0


In [29]:
Z <- snp_data
sigma_a_y1 = results_y1$sigma_a
sigma_e_y1 = results_y1$sigma_e
sigma_g_y1 = sigma_a / length(p2)
sigma_a_y2 = results_y2$sigma_a
sigma_e_y2 = results_y2$sigma_e
sigma_g_y2 = sigma_a / length(p2)
G = diag(length(p2))

ERROR: Error in eval(expr, envir, enclos): object 'sigma_a' not found
